In [ ]:
import numpy as np
from sklearn.metrics import f1_score, recall_score, accuracy_score
from collections import defaultdict
from sklearn.svm import LinearSVC, LinearSVR
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from sklearn.model_selection import cross_val_score, cross_validate, train_test_split

In [ ]:
data_path_vaccine = "/content/drive/MyDrive/Colab Notebooks/Capstone_govt_of_canada/data/Page feedback-Vaccine pages-May17.csv"
data_path_travel = "/content/drive/MyDrive/Colab Notebooks/Capstone_govt_of_canada/data/Page feedback-Travel-May17.csv"
data_path_benefits = "/content/drive/MyDrive/Colab Notebooks/Capstone_govt_of_canada/data/Page feedback-Benefits.csv"
data_path_taxes = "/content/drive/MyDrive/Colab Notebooks/Capstone_govt_of_canada/data/Page feedback-Taxes.csv"

In [ ]:
domains = ['vaccine', 'travel', 'benefits', 'taxes']
# df = pd.DataFrame(columns=['Unique ID', 'Date', 'Comment', 'Tags', 'Domain'])
df_lst = []
for domain in domains:
  exec(f'df_{domain} = pd.read_csv(data_path_{domain})')
  exec(f"df_{domain} = df_{domain}[df_{domain}['Tags confirmed']=='checked'][df_{domain}['Lang'] == 'EN']")
  exec(f"df_{domain}['Domain'] = '{domain}'")
  exec(f"df_lst.append(df_{domain}[['Unique ID', 'Date', 'Comment', 'Tags', 'Domain']].dropna())")
df = pd.concat(df_lst, axis=0, ignore_index=True)

In [ ]:
df

,Unique ID,Date,Comment,Tags,Domain
0,60a2746a2b6da5147c289ae3,"17 May, 2021",I want to see how many of the current covid ca...,Data and tracking vaccines,vaccine
1,60a2735c2b6da5147c289ada,"17 May, 2021",I wanted to know the number of deaths.,Data and tracking vaccines,vaccine
2,60a28aae2b6da5147c289b4f,"17 May, 2021",Are you hiding the deaths from vaccination?,Data and tracking vaccines,vaccine
3,60a28dd82b6da5147c289b66,"17 May, 2021",How come there are so few reported deaths in t...,Data and tracking vaccines,vaccine
4,60a220282b6da5147c289a3c,"17 May, 2021",Each province should state total vaccinations ...,Data and tracking vaccines,vaccine
...,...,...,...,...,...
44211,5fee454e8704dd24e00b9d71,"31 December, 2020",trying to register for on line T4E slip,T4 information (T1/T4/T4A/T4E/R1),taxes
44212,5fee4d648704dd24e00b9d83,"31 December, 2020",Where do 8 find the Application o file online?,Income tax form or guide,taxes
44213,5fee4eae8704dd24e00b9d85,"31 December, 2020",I need the on-line application form,Income tax form or guide,taxes
44214,5fee654c8704dd24e00b9da8,"31 December, 2020","I want to file on line , unable to find the re...",COVID benefits,taxes


In [ ]:
train_df, dev_df = train_test_split(df,  test_size=0.2, random_state=11)

In [ ]:
def prepare_for_classification(train,dev,max_n=2):
  '''convert lists of reviews train and dev to spare feature matrices X_train and X_test,
  and lists of polarity classifications train_class and dev_class'''
  vectorizer = CountVectorizer(ngram_range=(1,max_n),min_df=2)
  train_texts = train.Comment.values
  train_class = train.Domain.values
  dev_texts = dev.Comment.values
  dev_class = dev.Domain.values
  X_train = vectorizer.fit_transform(train_texts)
  X_dev = vectorizer.transform(dev_texts)
  return X_train,train_class, X_dev,dev_class

In [ ]:
def evluate(train, test, n = 2):
    """Calculate the the kendalltau score from given train test data set, and n grams"""
    
    X_train,train_class, X_test,test_class = prepare_for_classification(train,test,max_n=n)
    clf = LinearSVC()
    clf.fit(X_train,train_class)
    
    fscore = f1_score(test_class, clf.predict(X_test), average='macro')
    acc = accuracy_score(test_class, clf.predict(X_test))
    
    print(f"{n}-gram: The shape of training set is {X_train.shape}, the fscore is {fscore}, the accuracy is {acc}")

    return fscore
    
 
print("Domain:")
for n in range(1, 6):
    evluate(train_df,dev_df, n)

Domain:


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


1-gram: The shape of training set is (35372, 7368), the fscore is 0.8800040753751185, the accuracy is 0.9265038444142921
2-gram: The shape of training set is (35372, 46822), the fscore is 0.8793972322516708, the accuracy is 0.9276345545002261
3-gram: The shape of training set is (35372, 86980), the fscore is 0.8762268301788885, the accuracy is 0.9259384893713252
4-gram: The shape of training set is (35372, 110294), the fscore is 0.8786383546152217, the accuracy is 0.9269561284486658
5-gram: The shape of training set is (35372, 121251), the fscore is 0.8789474556703893, the accuracy is 0.9268430574400723
